# Exhibition - enriching information

In [1]:
import pandas as pd
import duckdb

In [2]:
df_exhibition = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_done/Artlas Exhibition.csv', dtype ='str')
df_catalog = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibition Catalog.csv', dtype ='str')
df_exhibition_section = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibition Section.csv', dtype ='str')
df_has_exhibited_in = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibited.csv', dtype ='str')
df_personage = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/personage/Exhibitor.csv', low_memory=False, dtype ='str')
df_artwork = pd.read_csv('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibitions_ids/Exhibited Work.csv', low_memory=False, dtype ='str')

In [3]:
df_catalog.rename(columns={'id': 'id_catalog'}, inplace=True)
df_exhibition.rename(columns={'id': 'id_exhibition'}, inplace=True)
df_exhibition_section.rename(columns={'id': 'id_section'}, inplace=True)
df_has_exhibited_in.rename(columns={'id_exhibition_section': 'id_section', 'id': 'id_has_exhibited_in'}, inplace=True)
df_artwork.rename(columns={'id': 'id_work'}, inplace=True)

In [7]:
df_personage.head(3)

,id_exhibition_section,id_personage,id,name,first_name,membership,biography,nationality,instructor,notes,...,country_d,x_address_d,y_address_d,x_city_d,y_city_d,gender_lower,address1,id_address1,address2,id_address2
0,3194,37218,33569,Manganaris,Yannis,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,m,NaN,NaN,NaN,NaN
1,3875,49029,41584,Abrams,Lionel,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,m,NaN,NaN,NaN,NaN
2,3875,49030,41585,Battiss,Walter,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,m,NaN,NaN,NaN,NaN


In [5]:
merged_catalog = pd.merge(df_exhibition[['id_exhibition']], df_catalog[['id_catalog', 'id_exhibition']], left_on='id_exhibition', right_on='id_exhibition')

In [6]:
merged_catalog.head(3)

,id_exhibition,id_catalog
0,8155,3697
1,8156,3698
2,18595,3908


In [8]:
merged_section = pd.merge(merged_catalog[['id_catalog', 'id_exhibition']], df_exhibition_section[['id_section', 'id_catalog']], left_on='id_catalog', right_on='id_catalog')

In [9]:
merged_section.head(3)

,id_catalog,id_exhibition,id_section
0,3697,8155,4007
1,3698,8156,4008
2,2675,2451,1482


In [10]:
merged_exhibited = pd.merge(merged_section[['id_catalog', 'id_exhibition', 'id_section']], df_has_exhibited_in[['id_personage', 'id_section', 'id_has_exhibited_in']], left_on='id_section', right_on='id_section')

In [11]:
merged_exhibited.head(3)

,id_catalog,id_exhibition,id_section,id_personage,id_has_exhibited_in
0,3697,8155,4007,50822,43377
1,3697,8155,4007,50823,43378
2,3697,8155,4007,50824,43379


In [12]:
merged_work = pd.merge(merged_exhibited[['id_catalog', 'id_exhibition', 'id_section','id_personage', 'id_has_exhibited_in']], df_artwork[['id_work', 'id_has_exhibited_in']], left_on='id_has_exhibited_in', right_on='id_has_exhibited_in')

In [13]:
merged_work.drop('id_has_exhibited_in', axis=1, inplace=True)

In [14]:
merged_work.head(3)

,id_catalog,id_exhibition,id_section,id_personage,id_work
0,3697,8155,4007,50822,102296
1,3697,8155,4007,50822,102297
2,3697,8155,4007,50822,102298


In [15]:
merged_addr = pd.merge(merged_work[['id_catalog', 'id_exhibition', 'id_section','id_personage', 'id_work']], df_personage[['id_personage', 'id_address1']], left_on='id_personage', right_on='id_personage')

In [16]:
merged_addr

,id_catalog,id_exhibition,id_section,id_personage,id_work,id_address1
0,3697,8155,4007,50822,102296,NaN
1,3697,8155,4007,50822,102297,NaN
2,3697,8155,4007,50822,102298,NaN
3,3697,8155,4007,50823,102299,NaN
4,3697,8155,4007,50824,102300,NaN
...,...,...,...,...,...,...
164652,4125,18773,4454,92718,257585,18336
164653,4125,18773,4454,92719,257586,15347
164654,4125,18773,4454,92719,257587,15347
164655,4125,18773,4454,92720,257588,NaN


# Saving the final results

In [19]:
columns = merged_addr.columns

# Iterate through the columns
for col in columns:
    # Check if the column name contains a space
    if ' ' in col:
        # Replace spaces with underscores
        merged_work.rename(columns={col: col.replace(' ', '_')}, inplace=True)

In [20]:
merged_addr.to_xml('/Users/carboni/Documents/UNIGE/basart_downloads/2023/data/processed/exhibition_extra/exhibition_enriched.xml', root_name="data", pretty_print=True)

# Data Analysis

Checking data consistency

## How many sections and catalogues per exhibition

In [ ]:
final_merged_with_exhibition = pd.merge(merged_work, df_exhibition, on='id_exhibition', how='left')

In [ ]:
duckdb.sql("CREATE TABLE exhibition AS SELECT * FROM final_merged_with_exhibition")

In [ ]:
result = duckdb.execute("SELECT * FROM exhibition").fetchdf()
result.head(5)

In [ ]:
query = """
SELECT id_exhibition, traveling, COUNT(DISTINCT id_section) as section_count, COUNT(DISTINCT id_catalog) as catalog_count_count
FROM exhibition
GROUP BY id_exhibition, traveling
HAVING COUNT(DISTINCT id_section) > 1;
"""

In [ ]:
result = duckdb.query(query).fetchdf()
print(result)

### Multiple catalogues' id example

In [ ]:
catalogues = duckdb.execute("SELECT DISTINCT id_section, id_catalog FROM exhibition WHERE id_exhibition = 18492;").fetchdf()

In [ ]:
print(catalogues)

### Multiple section example

In [ ]:
section = duckdb.execute("SELECT DISTINCT id_section, id_catalog FROM exhibition WHERE id_exhibition = 4283;").fetchdf()

In [ ]:
print(section)